<h4>TextCNN</h4>

<h5>Imports</h5>

In [49]:
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, unquote_plus
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Input, Dropout
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses
import jsonlines
import os
import json
import pickle
from sklearn.metrics import f1_score, accuracy_score, precision_score

In [2]:
file_names=['malicious','normal']
for name in file_names:
    data=[]
    with jsonlines.open('dataset/'+name+'.txt') as reader:
        for line in reader:
            if line['request']['method'] == 'POST':
                data.append({'request':{'method':'POST', 'uri':line['request']['uri'], 'body':line['request']['body'],'headers':line['request']['headers']},'metadata':line['metdata']})
            else:
                data.append({'request':{'method':'GET', 'uri':line['request']['uri'], 'headers':line['request']['headers']},'metadata':line['metdata']})
    with jsonlines.open('dataset/'+name+'_clean.txt', mode='w') as writer:
        writer.write_all(data)

<h4>Load and create datasets</h4>

In [33]:
file_names = ['dataset/malicious_clean.txt' ,'dataset/normal_clean.txt']
data={}
for file in file_names:
    data[file]=[]
    with jsonlines.open(file) as reader:
        for line in reader:
            if line['request']['method'] == 'POST':
                data[file].append('POST'+' '+line['request']['uri']+' '+line['request']['body']+' '+json.dumps(line['request']['headers']))
            else:
                data[file].append('GET'+' '+line['request']['uri']+' '+json.dumps(line['request']['headers']))
                
                
normal = data[file_names[1]]
malicious = data[file_names[0]]
normal_part1 = normal[180000:]
normal_part2 = normal[:180000]

train_examples = normal_part2
test_examples = normal_part1+malicious
train_labels = [0] * len(train_examples)
test_labels = [0]* len(normal_part1)
test_labels.extend([1] * len(malicious))

dataset_train = tf.data.Dataset.from_tensor_slices((list(map(lambda x: unquote_plus(x),train_examples)), train_labels))
dataset_test = tf.data.Dataset.from_tensor_slices((list(map(lambda x: unquote_plus(x),test_examples)), test_labels))


def preprocess_text(text, label):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "[a-zA-Z]+", "a")
    text = tf.strings.regex_replace(text, "[0-9]+", "n")
    text = tf.strings.regex_replace(text, "(a|n){2,}", "x")
    text = tf.strings.regex_replace(text, '[^\x00-\x7F]+', '')
    text = tf.strings.regex_replace(text, '(.)',  r'\1 ')
    return text, label

def preprocess_text_substitution(text, label):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "[a-zA-Z]+", "a")
    text = tf.strings.regex_replace(text, "[0-9]+", "n")
    text = tf.strings.regex_replace(text, "(a|n){2,}", "x")
    text = tf.strings.regex_replace(text, '[^\x00-\x7F]+', '')
    punctuation = "=?/(){}[]<>"
    for p in punctuation:
        text = tf.strings.regex_replace(text, "\\" + p, " "+p+" ")
    return text, label

 
# Map the preprocess function to the dataset
train = dataset_train.map(preprocess_text)
test = dataset_test.map(preprocess_text)

train_sub = dataset_train.map(preprocess_text_substitution)
test_sub = dataset_test.map(preprocess_text_substitution)

for text, label in train_sub.take(1):
    print(text.numpy(), label.numpy())

for text, label in train.take(1):
    print(text.numpy(), label.numpy())

b'a  / a / a / a  { "a" = "a" }   { "a-a": "a", "a": "a / a,* / *", "a-a": "a, a, a", "a-a": "a / n.n  ( a a n.n; x; x )  a / n.n  ( a, a a )  a / n.n.n.n a / n.n", "a": "a = x; a = x" } ' 0
b'a   / a / a / a   { " a " = " a " }   { " a - a " :   " a " ,   " a " :   " a / a , * / * " ,   " a - a " :   " a ,   a ,   a " ,   " a - a " :   " a / n . n   ( a   a   n . n ;   x ;   x )   a / n . n   ( a ,   a   a )   a / n . n . n . n   a / n . n " ,   " a " :   " a = x ;   a = x " } ' 0


2023-04-23 00:43:23.778384: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [180000]
	 [[{{node Placeholder/_1}}]]
2023-04-23 00:43:23.799568: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [180000]
	 [[{{node Placeholder/_1}}]]


<h4>Vocabularies</h4>

In [34]:

vectorization = TextVectorization(
    max_tokens=1000,
    output_mode="int", 
    output_sequence_length=256,
    pad_to_max_tokens=True,
    standardize="lower"
)

vectorization_sub = TextVectorization(
    max_tokens=5000, 
    output_mode="int", 
    output_sequence_length=128,
    pad_to_max_tokens=True,
    standardize="lower"
)

vectorization.adapt(train.map(lambda x, y: x))
vectorization_sub.adapt(train_sub.map(lambda x, y: x))

2023-04-23 00:43:23.850876: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [180000]
	 [[{{node Placeholder/_1}}]]
2023-04-23 00:53:16.487408: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [180000]
	 [[{{node Placeholder/_1}}]]


<h4>Save and load vectorization from disk</h4>

In [35]:
pickle.dump({'config': vectorization.get_config(),'weights': vectorization.get_weights()}, open("vectorization.pkl", "wb"))

pickle.dump({'config': vectorization_sub.get_config(), 'weights': vectorization_sub.get_weights()}, open("vectorization_sub.pkl", "wb"))


from_disk = pickle.load(open("vectorization.pkl", "rb"))
vectorization = TextVectorization.from_config(from_disk['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
vectorization.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
vectorization.set_weights(from_disk['weights'])

from_disk_sub = pickle.load(open("vectorization_sub.pkl", "rb"))
vectorization_sub = TextVectorization.from_config(from_disk_sub['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
vectorization_sub.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
vectorization_sub.set_weights(from_disk_sub['weights'])

2023-04-23 01:01:40.649481: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2023-04-23 01:01:40.737564: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]


<h4>Autoencoder</h4>

In [36]:
embedding_layer = tf.keras.layers.Embedding(1000, 4)
flatten_layer = tf.keras.layers.Flatten()

def autoencoder_processing(text, label):
    return  vectorization(text), vectorization(text)

train_auto = train_sub.map(autoencoder_processing)

train_auto = train_auto.batch(32)
# Define the autoencoder model
input_layer = tf.keras.layers.Input(shape=(256,))
encoder = tf.keras.layers.Dense(256, activation='relu')(input_layer)
encoder = tf.keras.layers.Dense(128, activation='relu')(encoder)
encoder = tf.keras.layers.Dense(64, activation='relu')(encoder)
decoder = tf.keras.layers.Dense(128, activation='relu')(encoder)
decoder = tf.keras.layers.Dense(256, activation='relu')(decoder)
autoencoder = tf.keras.models.Model(input_layer, decoder)

# Compile and train the model
autoencoder.compile(optimizer='adam',  loss=losses.MeanSquaredError())
autoencoder.fit(train_auto, epochs=10)

Epoch 1/10


2023-04-23 01:02:31.433838: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [180000]
	 [[{{node Placeholder/_0}}]]


5625/5625 [==============================] - 46s 8ms/step - loss: 0.7587
Epoch 2/10
5625/5625 [==============================] - 42s 7ms/step - loss: 0.4940
Epoch 3/10
5625/5625 [==============================] - 40s 7ms/step - loss: 0.4653
Epoch 4/10
5625/5625 [==============================] - 40s 7ms/step - loss: 0.4502
Epoch 5/10
5625/5625 [==============================] - 40s 7ms/step - loss: 0.4398
Epoch 6/10
5625/5625 [==============================] - 43s 8ms/step - loss: 0.4331
Epoch 7/10
5625/5625 [==============================] - 40s 7ms/step - loss: 0.4285
Epoch 8/10
5625/5625 [==============================] - 39s 7ms/step - loss: 0.4254
Epoch 9/10
5625/5625 [==============================] - 40s 7ms/step - loss: 0.4235
Epoch 10/10
5625/5625 [==============================] - 40s 7ms/step - loss: 0.4203


In [37]:
#autoencoder.fit(train_auto, epochs=10)
autoencoder.save('autoencoder/my_model')
autoencoder = tf.keras.models.load_model('autoencoder/my_model')


INFO:tensorflow:Assets written to: autoencoder/my_model/assets


In [38]:
def autoencoder_processing(text, label):
    return  vectorization(text), label
test_auto = test_sub.map(autoencoder_processing)
test_auto = test_auto.batch(32)
reconsturctions =  autoencoder.predict(test_auto)

   8/3685 [..............................] - ETA: 30s 

2023-04-23 01:10:41.067879: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype string
	 [[{{node Placeholder/_6}}]]


3685/3685 [==============================] - 24s 6ms/step


In [39]:
def get_data(x, y):
  return x

# Use the map() function to extract the labels
dataset_tmp = test_auto.map(get_data)

# Convert the labels dataset to a NumPy array
tmp = np.concatenate(list(dataset_tmp.as_numpy_iterator()))
mse = tf.keras.losses.mean_squared_error(tmp, reconsturctions)

2023-04-23 01:11:06.176232: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [117899]
	 [[{{node Placeholder/_0}}]]


In [45]:
pred = list(map(lambda x: 0 if x.numpy()<0.09 else 1, list(mse)))

In [59]:
actual =test_auto.map(lambda x,y:y)
actual = np.concatenate(list(actual.as_numpy_iterator()))


2023-04-23 01:28:53.801071: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype string
	 [[{{node Placeholder/_6}}]]


In [62]:
f1_score(actual,pred)

0.8628199463933316